In [8]:
%load_ext autoreload
%autoreload 2

from integ_auto import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

from integ_auto import *

# def _create_driver(headless=False):
#     options = webdriver.EdgeOptions()
#     options.add_argument('log-level=3')
#     if headless:
#         options.add_argument('headless')
#         options.add_argument('disable-gpu')

#     service = Service(EdgeChromiumDriverManager().install())
#     return webdriver.Edge(options=options, service=service)

# driver = _create_driver()

driver = Automatic.create_edge_driver()
auto = Automatic(driver)

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 104.0.1293
[WDM] - Get LATEST edgedriver version for 104.0.1293 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/104.0.1293.54/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Jaeyoung\.wdm\drivers\edgedriver\win64\104.0.1293.54]


In [2]:


def _pre_login(auto:Automatic):
    # 1. 홈페이지로 이동
    auto.go("https://bid.kogas.or.kr:9443/supplier/index.jsp") 

    # 2. 공인인증서 로그인 버튼 
    login_btn = auto.get_element(By.XPATH, '//img[@alt="공동인증서로그인"]')
    if not login_btn:
        print("Error: Failed to find 로그인 button")
        return False
    auto.click(login_btn)

    # 3. 인증서 선택창
    # 3.1 지문보안토큰 클릭
    bio_token = auto.get_element(By.ID, "NX_MEDIA_BIOHSM")
    if not bio_token:
        print("Error: Failed to find 지문보안토큰")
        return False
    auto.click(bio_token)


    bio_seal = auto.get_element(By.XPATH, '//*[@id="(주)유니온커뮤니티 BIO-SEAL|FP_HSM.dll|1.0.2.1|설치"]')
    if not bio_seal:
        print("Error: Failed to find BIO-SEAL 보안 토큰")
        return False
    auto.click(bio_seal)

    return True


    
_pre_login(auto)
    

True

In [3]:

def _login(auto:Automatic):

    # 토큰 선택 확인
    token_sel_confirm_btn = auto.get_element(By.XPATH, '//*[@id="pki-extra-media-box-contents3"]/div[3]/button[1]')
    if not token_sel_confirm_btn:
        print("Error: Failed to find 토큰 선택 확인")
        return False
    auto.click(token_sel_confirm_btn)

    # User Interaction needed

    # 4. pin 번호 입력
    # pin번호 입력
    input = auto.get_clickable(By.ID, "nx_cert_pin", timeout=120)
    if not input:
        print("Failed to find input field for the pin")
        return False
    auto.type(input, "00000000")

    # pin 번호 입력 확인 버튼 
    pin_confirm_btn = auto.get_element(By.XPATH, '//*[@id="pki-extra-media-box-contents3"]/div[2]/button[1]')
    if not pin_confirm_btn:
        return False
    auto.click(pin_confirm_btn)

    # 인증서 선택 확인 버튼 
    cert_confirm_btn = auto.get_element(By.XPATH, '//*[@id="nx-cert-select"]/div[4]/button[1]')
    if not cert_confirm_btn:
        return False
    auto.click(cert_confirm_btn)

    return True


_login(auto)
    

True

In [11]:


# register

def _go_detail_page(auto:Automatic, number):
    top_frame = auto.get_element(By.XPATH, "//frame[@src='/supplier/top.jsp']")
    if not top_frame:
        print("Failed to find top frame")
        return False
    with auto.get_frame(top_frame):
        # 1. 전자 입찰 버튼 클릭
        if not auto.click(By.ID, 'tm_01'):
            print("Failed to find 전자 입찰 버튼 클릭")
            return False

    body_frame = auto.get_element(By.XPATH, "//frame[@src='/supplier/bodyframe.jsp']")
    if not body_frame:
        print("Failed to find bodyframe frame")
        return False
    with auto.get_frame(body_frame):
        body = auto.get_element(By.XPATH, "//frame[@src='/supplier/body.jsp']")
        if not body:
            print("Failed to find body frame")
            return False
        with auto.get_frame(body):
            # 입찰번호 입력
            if not auto.type(By.XPATH, '//input[@title="입찰번호"]', number):
                return False
            
            # 검색 버튼 클릭
            if not auto.click(By.XPATH, '//img[@alt="검색"]'):
                return False

            # 검색 결과 확인
            num = auto.get_element(By.XPATH, '//tbody/tr[2]/td[1]/a')
            if not num:
                print(f"Failed to find the item. number={number}")
                return False

            if num.text != number:
                print(f"Faile to validate number. expected={number}, real={num}")
                return False

            # 첫번째 item 선택
            if not auto.click(By.XPATH, '//tbody/tr[2]/td[2]/a/span'):
                return False

          

    return True


# # Precondition:
# #  - detail page
# def _is_registered():

def close_other_windows(driver:WebDriver):
    current = driver.current_window_handle
    handles = driver.window_handles
    handles.remove(current)
    for handle in handles:
        driver.switch_to.window(handle)
        driver.close()
    driver.switch_to.window(current)


def register(auto:Automatic, number):

    # go homepage
    auto.go("https://bid.kogas.or.kr:9443/supplier/index.jsp")
    
    close_other_windows(auto.driver)

    # number requirement: 뒤에 번호는 제거
    # 2022062215001-00 -> 2022062215001
    number = number.split('-')[0]

    if not _go_detail_page(auto, number):
        print(f"Failed to go to the detail page. number={number}")
        return False


    def work_in_frame(func):
        body_frame = auto.get_element(By.XPATH, "//frame[@src='/supplier/bodyframe.jsp']")
        if not body_frame:
            print("Failed to find bodyframe frame")
            return False
        with auto.get_frame(body_frame):
            body = auto.get_element(By.XPATH, "//frame[@src='/supplier/body.jsp']")
            if not body:
                print("Failed to find body frame")
                return False
            with auto.get_frame(body):
                return func()

    
    # WHAT TO DO: 입찰참가신청
    def register_lv1():
        # # 입찰 참가신청
        # if not auto_click(driver, By.XPATH, '//a[text()="입찰참가신청"]'):
        # 입찰참가신청 탭으로 이동
        if not auto.click(By.XPATH, '//*[@id="tab_name_2"]/a'):
            return False

        # 입찰당당자 입력
        auto.type(By.XPATH, '//input[@name="ca_name"]', '박재영')
        auto.type(By.XPATH, '//input[@name="ca_tel"]', '01031084585')
        auto.type(By.XPATH, '//input[@name="ca_email"]', 'jaeyoungs.park@gmail.com')

        # 입찰참가신청 동의 버튼
        if not auto.click(By.XPATH, '//*[@id="registpanel"]/table[17]/tbody/tr/td/a[1]/img'):
            return False

        # Alert 동의 - 약관에 동의하시겠습니까?
        alert = auto.get_alert()
        print(alert.text)

        if not auto.accept_alert():
            print("Error: Failed to accept alert. - 약관 동의")
            return False                

        # 동의 버튼 클릭
        if not auto.click(By.XPATH, '//*[@id="registpanel"]/table[2]/tbody/tr[9]/td[3]/a/img'):
            return False

    def register_lv2():
        # 입찰보증금지급각서 윈도우로 이동하여 동의 버튼 
        handle = auto.get_window_handle("한국가스공사 전자조달시스템")
        if not handle:
            return False                
        with auto.get_window(handle):
            if not auto.click(By.XPATH, '//img[@alt="동의"]'):
                return False

            # 입찰보증금 지급각서에 동의 하시겠습니까?
            if not auto.accept_alert():
                return False

    def register_lv3():           
        # 제출 버튼 클릭
        if not auto.click(By.XPATH, '//*[@id="registpanel"]/table[3]/tbody/tr/td/a[1]/img'):
            print("Error: Failed to find 제출 버튼")
            return False

            # alert = auto.get_alert()
            # print(alert.text)

            # if not auto.accept_alert():
            #     print("Error: Failed to accept alert. ")
            #     return False           

    work_in_frame(lambda: register_lv1())
    register_lv2()
    work_in_frame(lambda: register_lv3())

    return True
    

register(auto, "2022080912001-00")









약관에 동의하시겠습니까?


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: MicrosoftEdge=104.0.1293.54)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::unpack [0x00007FF6BFE341A2+26370]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00007FF6BFD89502+339106]
	Ordinal0 [0x00007FF6BF917821+620577]
	Ordinal0 [0x00007FF6BF91AD9A+634266]
	Ordinal0 [0x00007FF6BF91AB1B+633627]
	Ordinal0 [0x00007FF6BF91AED0+634576]
	Ordinal0 [0x00007FF6BF954C90+871568]
	Ordinal0 [0x00007FF6BF9562E3+877283]
	Ordinal0 [0x00007FF6BF984E7E+1068670]
	Ordinal0 [0x00007FF6BF96F44A+980042]
	Ordinal0 [0x00007FF6BF98358F+1062287]
	Ordinal0 [0x00007FF6BF96F263+979555]
	Ordinal0 [0x00007FF6BF946070+811120]
	Ordinal0 [0x00007FF6BF947668+816744]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6BFBE2998+128088]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6BFBCC2EB+36267]
	Microsoft::Applica

In [6]:


current = driver.current_window_handle
hs = driver.window_handles
for h in hs:
    driver.switch_to.window(h)
    print(driver.current_url)
    print(driver.title)
driver.switch_to.window(current)


https://bid.kogas.or.kr:9443/supplier/index.jsp
한국가스공사 전자조달시스템
https://bid.kogas.or.kr:9443/supplier/bid/bid_detail_view_bidguar_popup.jsp?notice_code=2022080912&bid_code=001&round=01
한국가스공사 전자조달시스템


In [27]:


def register(driver:WebDriver, number, price):
    
    _search_item(driver, number)

    status = wait_element(driver, By.XPATH, '//table[3]/tbody/tr[2]/td[8]')
    if not status:
        print("Failed to find item status")
        return False

    if status.text != "완료":
        print(f"Failed to validate item status. expected=완료, real={status.text}")
        return False

    # 아이템 디테일 페이지로 이동
    
    # 입찰서 제출 페이지로 이동
    auto_click(driver, By.ID, '//a[text()="입찰서제출"]')

    

    return False
